In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
delaney_chemberta2_valid2 = pd.read_csv('./chemberta2/results/delaney/chemberta2_valid2_delaney_2_predictions.csv')
delaney_molformer_valid2 = pd.read_csv('./molformer/results/delaney/molformer_valid2_delaney_2_99.csv')
delaney_molbert_valid2 = pd.read_csv('./molbert/results/delaney/molbert_valid2_delaney_2.csv')

# Load the test data for each model
delaney_chemberta2_test = pd.read_csv('./chemberta2/results/delaney/chemberta2_test_delaney_2_predictions.csv')
delaney_molformer_test = pd.read_csv('./molformer/results/delaney/molformer_test_delaney_2_99.csv')
delaney_molbert_test = pd.read_csv('./molbert/results/delaney/molbert_test_delaney_2.csv')

train_mean = -3.0330665680473374
train_sd = 2.0662151693210706

# features

# Load the features from chemberta
delaney_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/delaney/chemberta2_valid2_delaney_2_features.csv')
delaney_chemberta2_features_test = pd.read_csv('./chemberta2/features/delaney/chemberta2_test_delaney_2_features.csv')

# Load the features from molformer
delaney_molformer_features_valid2 = pd.read_csv('./molformer/features/delaney/molformer_valid2_delaney_2_features.csv')
delaney_molformer_features_test = pd.read_csv('./molformer/features/delaney/molformer_test_delaney_2_features.csv')

# Load the features from molbert
delaney_molbert_features_valid2 = pd.read_csv('./molbert/features/delaney/molbert_valid2_delaney_2_features.csv')
delaney_molbert_features_test = pd.read_csv('./molbert/features/delaney/molbert_test_delaney_2_features.csv')

For delaney (regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
delaney_chemberta_actual = delaney_chemberta2_test['target'] 
delaney_chemberta_pred = delaney_chemberta2_test['pred_raw']

# Molformer
delaney_molformer_actual = delaney_molformer_test['target']
delaney_molformer_pred = delaney_molformer_test['pred_raw']

# molbert
delaney_molbert_actual = delaney_molbert_test['target_raw']
delaney_molbert_pred = delaney_molbert_test['pred_raw']

In [3]:
# Calculating metrics
delaney_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", delaney_chemberta_actual, delaney_chemberta_pred),
                                 ("Molformer", delaney_molformer_actual, delaney_molformer_pred),
                                 ("Molbert", delaney_molbert_actual, delaney_molbert_pred)]:
    delaney_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

delaney_metrics_results

{'Chemberta2': {'MAE': 0.4969548122463181,
  'RMSE': 0.6739879851585397,
  'R2 Score': 0.8995271768026132,
  'Correlation': 0.9520502440343135},
 'Molformer': {'MAE': 0.47515824184070793,
  'RMSE': 0.6199320972655071,
  'R2 Score': 0.9149973350094271,
  'Correlation': 0.9664923764380632},
 'Molbert': {'MAE': 0.39575742491150445,
  'RMSE': 0.5147120546385228,
  'R2 Score': 0.9414033246688129,
  'Correlation': 0.9711342185240168}}

In [4]:
# standardized valid2 labels
delaney_y_ensemble_valid2 = (delaney_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
delaney_X_ensemble_valid2 = pd.concat([
    delaney_chemberta2_valid2['pred_z'] - delaney_y_ensemble_valid2,
    delaney_molformer_valid2['pred_z'] - delaney_y_ensemble_valid2, 
    delaney_molbert_valid2['pred_z'] - delaney_y_ensemble_valid2,
    # add features from training set
    delaney_chemberta2_valid2['pred_z'],
    delaney_molformer_valid2['pred_z'],
    delaney_molbert_valid2['pred_z']
], axis=1)

# change feature names of the ensemble so that they are unique
delaney_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
seed = 0

# Compute residuals
chemberta_y_residual = delaney_chemberta2_valid2['pred_z'] - delaney_y_ensemble_valid2
molformer_y_residual = delaney_molformer_valid2['pred_z'] - delaney_y_ensemble_valid2
molbert_y_residual = delaney_molbert_valid2['pred_z'] - delaney_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([delaney_chemberta2_valid2['pred_z'], 
                         delaney_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([delaney_molformer_valid2['pred_z'], 
                         delaney_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([delaney_molbert_valid2['pred_z'], 
                       delaney_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Initialize the ElasticNet models with a random_state for reproducibility
elastic_net_chemberta = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)
elastic_net_molformer = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)
elastic_net_molbert = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)

# Setup cross-validation for alpha and l1_ratio tuning
param_grid = {
    'alpha': np.logspace(-4, 1, 10),
    'l1_ratio': np.linspace(0, 1, 10)  # l1_ratio between 0 (Ridge) and 1 (Lasso)
}

# Set up GridSearchCV for ElasticNet with neg_mean_squared_error scoring and random_state
cv_chemberta = GridSearchCV(estimator=elastic_net_chemberta, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molformer = GridSearchCV(estimator=elastic_net_molformer, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molbert = GridSearchCV(estimator=elastic_net_molbert, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV for each dataset with residual as the target
cv_chemberta.fit(chemberta_X_scaled, chemberta_y_residual)
cv_molformer.fit(molformer_X_scaled, molformer_y_residual)
cv_molbert.fit(molbert_X_scaled, molbert_y_residual)

# Retrieve the best models and parameters
best_model_chemberta = cv_chemberta.best_estimator_
best_model_molformer = cv_molformer.best_estimator_
best_model_molbert = cv_molbert.best_estimator_

/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.988e-02, tolerance: 2.124e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.867e-02, tolerance: 1.991e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers 

In [6]:
# ChemBERTa
chemberta_X_test = pd.concat([delaney_chemberta2_test['pred_z'], 
                              delaney_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([delaney_molformer_test['pred_z'], 
                              delaney_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([delaney_molbert_test['pred_z'], 
                            delaney_molbert_features_test.iloc[:, 1:]], axis=1)

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict residuals for the test set using the trained GroupLasso models
y_pred_residuals_chemberta_test = best_model_chemberta.predict(chemberta_X_test_scaled)
y_pred_residuals_molformer_test = best_model_molformer.predict(molformer_X_test_scaled)
y_pred_residuals_molbert_test = best_model_molbert.predict(molbert_X_test_scaled)

# Create a DataFrame to store the predicted residuals for the test set
predicted_residuals_test = pd.DataFrame({
    'Residuals_Chemberta': y_pred_residuals_chemberta_test,
    'Residuals_Molformer': y_pred_residuals_molformer_test,
    'Residuals_Molbert': y_pred_residuals_molbert_test
})

# Load test data into ensemble DataFrame, adding the predicted residuals and original predictions
delaney_X_ensemble_test = pd.concat([
    predicted_residuals_test,
    delaney_chemberta2_test['pred_z'],
    delaney_molformer_test['pred_z'],  
    delaney_molbert_test['pred_z']
], axis=1)

# Rename feature columns so that they are unique
delaney_X_ensemble_test.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

# True test labels
delaney_y_ensemble_test = delaney_chemberta2_test['target']

In [7]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
delaney_X_ensemble_valid2_scaled = scaler.fit_transform(delaney_X_ensemble_valid2)
delaney_X_ensemble_test_scaled = scaler.transform(delaney_X_ensemble_test)

delaney_X_ensemble_valid2_scaled = pd.DataFrame(delaney_X_ensemble_valid2_scaled, columns=delaney_X_ensemble_valid2.columns)
delaney_X_ensemble_test_scaled = pd.DataFrame(delaney_X_ensemble_test_scaled, columns=delaney_X_ensemble_test.columns)

In [8]:
# elastic net
# Define the model with elasticnet penalty for regression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

elastic_net_model = ElasticNet(random_state=0, max_iter=5000)

# Define the hyperparameter grid
# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

params = {
    'alpha': alphas,  # Convert alpha back to C
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Get the best hyperparameters
delaney_best_elastic_params = grid_search.best_params_
print(delaney_best_elastic_params)

# Initialize and train the best ElasticNet model
delaney_best_elastic_model = ElasticNet(alpha=delaney_best_elastic_params['alpha'], l1_ratio=delaney_best_elastic_params['l1_ratio'], random_state=0, max_iter=5000)
delaney_best_elastic_model.fit(delaney_X_ensemble_valid2_scaled, delaney_y_ensemble_valid2)

# Predict the test set
delaney_elastic_pred = delaney_best_elastic_model.predict(delaney_X_ensemble_test_scaled) * train_sd + train_mean

# Calculate the metrics
delaney_elastic_metrics = {
    "MAE": mean_absolute_error(delaney_y_ensemble_test, delaney_elastic_pred),
    "RMSE": np.sqrt(mean_squared_error(delaney_y_ensemble_test, delaney_elastic_pred)),
    "R2 Score": r2_score(delaney_y_ensemble_test, delaney_elastic_pred),
    "Correlation": pearsonr(delaney_y_ensemble_test, delaney_elastic_pred)[0]
}

delaney_elastic_metrics

{'alpha': 0.01, 'l1_ratio': 0.1}


{'MAE': 0.37317173229050654,
 'RMSE': 0.49373391050567017,
 'R2 Score': 0.9460824426309424,
 'Correlation': 0.9738695517398958}

In [10]:
# create a table to record all metrics for delaney
delaney_metrics_results["Elastic Net"] = delaney_elastic_metrics

delaney_metrics_df = pd.DataFrame(delaney_metrics_results).T
# keep 3 digits after the decimal point
delaney_metrics_df = delaney_metrics_df.round(3)

# export table to csv
delaney_metrics_df.to_csv('./split2_delaney_metrics_elastic.csv')